<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/HW/%5BHW11%5DMultiple_linear_regression_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#이번에는 2개 이상의 독립변수 x에 대해 하나의 종속변수 y의 관계를 알아보자.
#이것을 Multiple linear regression이라고 한다. 

In [ ]:
#data set
#사용할 데이터셋은 auto miles per gallons(MPG)로
# 자동차의 여러 기술적인 사양들을 고려하여 연비를 예측하고자 한다. 

#library load
import pandas as pd
import seaborn                  #matplotlib을 기반으로 다양한 색상 테마와 통계용 차트 등의 기능을 추가한 시각화 패키지.
seaborn.set()                   #스타일 지정

In [ ]:
#data download
from urllib.request import urlretrieve
URL = 'https://go.gwu.edu/engcomp6data3'
urlretrieve(URL, 'auto_mpg.csv')

In [ ]:
#data load and check
mpg_data = pd.read_csv('/content/auto_mpg.csv')
mpg_data.head()

In [ ]:
mpg_data.info()                  #data info check

In [ ]:
#여기서 car name은 object형이고 origin은 categorical이다. 
#그러므로 여기서 두 변수를 제외하고 생각해보자. 
#target은 mpg이다.
y_col = 'mpg'
x_cols = mpg_data.columns.drop(['car name', 'origin', 'mpg'])

print(x_cols)

In [ ]:
#본격적으로 분석을 진행하기 전에 자동차의 정보들과 연비(mpg)의 1:1 상관관계를 알아보자.
#시각화하여 직관적으로 이해하자.
seaborn.pairplot(data=mpg_data, height=5, aspect=1,
                 x_vars = x_cols,
                 y_vars = y_col);                       #accerlation과 model_year의 정보는 양의 상관관계를, 나머지는 음의 상관관계가 있다. 
#결과적으로 상관관계를 통해 linear model이 연비를 예측하는데 충분하다는 것을 알 수 있다.                 

In [ ]:
#Linear Model in Matrix form
from autograd import numpy 
from autograd import grad

In [ ]:
X = mpg_data[x_cols].values
X = numpy.hstack((numpy.ones((X.shape[0],1)), X))   #X의 맨 왼족에 1로만 이루진 벡터를 추가한다. 
y = mpg_data[y_col].values

print("X.shape = {}, y.shape = {}".format(X.shape, y.shape))

In [ ]:
#cost function define -> MSE

def linear_regression(params, X):
    '''
    The linear regression model in matrix form. 
    Arguments:
      params: 1D array of weights for the linear model
      X     : 2D array of input values
    Returns:
      1D array of predicted values
    '''
    return numpy.dot(X, params)                            #numpy array를 곱할 때 사용.

def cost_function(params, model, X, y):
    '''
    The mean squared error loss function.
    Arguments:
      params: 1D array of weights for the lienar model
      model : function for the linear regression model
      X     : 2D array of input values
      Y     : 1D array of predicted values
    Returns:
      float, mean squeared error
    '''
    y_pred = model(params, X)
    return numpy.mean(numpy.sum((y-y_pred)**2))

In [ ]:
#gradient descent를 사용하여 weights를 찾아보자
#cost function을 정의했으니 gradient descent를 사용하여 cost를 최소로 하는 계수를 찾아보자. 

#1.autograd.grad()를 사용하여 기울기를 구해보자. 
#여기서 구해지는 기울기가 해당 변수의 weight이다.
gradient = grad(cost_function)                   #cost_function의 기울기를 구하는 기능을 만들어보자. 

In [ ]:
#기울기 확인
gradient(numpy.random.rand(X.shape[1]), linear_regression, X, y)          #기울기가 크게 나온다. 다시 진행하자.

In [ ]:
max_iter = 30     #반복 횟수
alpha = 0.001     #learning rate
params = numpy.zeros(X.shape[1])      #각 weight의 임의의 값을 0으로 설정.

for i in range(max_iter):
    descent = gradient(params, linear_regression, X, y)
    params = params - descent * alpha                        #params 수정
    loss = cost_function(params, linear_regression, X, y)    #해당 params로 구한 cost
    if i%5 == 0:
        print("iteration {}, loss = {}".format(i, loss))

In [ ]:
#loss가 무한대로 넘어가서 오류가 발생했다. 즉, loss가 줄어들지 않고 증가했다. 
#왜 이런 문제가 발생했을까?
#-> input value 중 특정 값들이 너무 커서 일어난 일이다. input value의 max, min값을 확인해보자.
mpg_data[x_cols].describe().loc[['max','min']]    #변수들 사이의 scale차이가 크다. 정규화가 필요하다!!!

In [ ]:
#우리는 변수 정규화의 방법으로 min-max scalng을 사용할 것이다. 
#간편하게 scikit-learn 패키지를 사용해보자.
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(mpg_data[x_cols])
X_scaled = numpy.hstack((numpy.ones((X_scaled.shape[0],1)), X_scaled))

In [ ]:
pd.DataFrame(X_scaled).describe().loc[['max','min']]              #데이터가 정규화되었다!!!

In [ ]:
#이제 다시 gradient descent를 진행해보자.
max_iter = 1000
alpha = 0.001
params = numpy.zeros(X.shape[1])

for i in range(max_iter):
    descent = gradient(params, linear_regression, X_scaled, y)
    params = params - descent * alpha
    loss = cost_function(params, linear_regression, X_scaled, y)
    if i%100 == 0:
        print("iteration {}, loss = {}".format(i, loss))

In [ ]:
#params의 학습이 완료되었다. 학습된 params는 다음과 같고,
#우리는 예측값을 params와 X의 곱으로 나타낼 수 있다.
params

In [ ]:
y_pred_gd = X_scaled @ params

In [ ]:
#우리가 만든 모델이 얼마나 정확한지 확인해보자
#주로 사용하는 지표는 MAE와 RMSE이다. scikit-learn을 사용하여 확인해보자.
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y, y_pred_gd)
rmse = mean_squared_error(y, y_pred_gd, squared=False)
print('MAE = {}'.format(mae))                           #2.61
print('RMSE = {}'.format(rmse))                         #3.41

Gradient descent land global temperature anomlay dataset


In [ ]:
#single linear regression을 공부할 때 사용했던 데이터를 불러와서 실습해보자.
from urllib.request import urlretrieve
URL = 'http://go.gwu.edu/engcomp1data5?accessType=DOWNLOAD'
urlretrieve(URL, 'land_global_temperature_anomaly-1880-2016.csv')

In [ ]:
import numpy as np
import pandas as pd
import sympy

In [ ]:
fname = '/content/land_global_temperature_anomaly-1880-2016.csv'

#txt 데이터 불러오기
year, temp_anomaly = np.loadtxt(fname, delimiter=',', skiprows=5, unpack=True)

In [ ]:
temp_anomaly

In [ ]:
#시각화하여 확인해보자.
from matplotlib import pyplot
%matplotlib inline

In [ ]:
pyplot.rc('font', family = 'serif', size = '18')

#이미지 사이즈 설정
pyplot.figure(figsize = (10,5))

#plot을 그려보자
pyplot.plot(year, temp_anomaly, color = '#2929a3', linestyle = '-', linewidth=1)
pyplot.title('Land global temperature anomalies, \n')
pyplot.xlabel('Year')
pyplot.ylabel('Land temperature anomaly [ C]')
pyplot.grid();                                            #grid를 없애줌

In [ ]:
#gradient descent 방법으로 진행해보자.

#먼저 cost function을 정의해보자
w, b, x, y = sympy.symbols('w b x y')

cost_function = (w*x + b - y)**2
cost_function

In [ ]:
grad_b = sympy.lambdify([w,b,x,y], cost_function.diff(b), 'numpy')
grad_w = sympy.lambdify([w,b,x,y], cost_function.diff(w), 'numpy')

In [ ]:
w=0
b=0

for i in range(1000):
    descent_b = np.sum(grad_b(w,b,year,temp_anomaly))/len(year)
    descent_w = np.sum(grad_w(w,b,year,temp_anomaly))/len(year)
    w = w - descent_w*0.001
    b = b - descent_b*0.001

print(w)
print(b)

In [ ]:
#params가 발산했다.이번에도 scaleing이 필요하다. 
#이번에는 mean normalization을 사용해보자.
year_scaled = (year - year.mean())/year.std()
temp_anomaly_scaled = (temp_anomaly - temp_anomaly.mean()) / temp_anomaly.std()

In [ ]:
#이제 다시 gradient descent를 진행해보자.
w=0
b=0

for i in range(1000):
    descent_b = np.sum(grad_b(w,b,year_scaled,temp_anomaly_scaled))/len(year_scaled)
    descent_w = np.sum(grad_w(w,b,year_scaled,temp_anomaly_scaled))/len(year_scaled)
    w = w - descent_w*0.001
    b = b - descent_b*0.001

print(w)                      #0.75
print(b)                      #5.55

In [ ]:
#params를 전부 구했다. 이제 다시 시각화해보자.
reg = b + w*year_scaled

In [ ]:
pyplot.plot(year_scaled, temp_anomaly_scaled, color='#2929a3', linestyle='-', linewidth=1, alpha=0.5) 
pyplot.plot(year_scaled, reg, 'k--', linewidth=2, label='Linear regression')
pyplot.xlabel('Year')
pyplot.ylabel('Land temperature anomaly [°C]')
pyplot.legend(loc='best', fontsize=15)
pyplot.grid();